In [1]:
import re, csv, numpy as np, sys, pandas as pd
from keras import models
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

In [2]:
skip = True

with open('adult/adult_after.test','w', encoding='utf-8') as csvfile:
    with open('adult/adult.test', 'r', newline='') as filein:
        for line in filein:
            if skip:
                skip=False
                continue
            flage = 0
            for i in range(1,len(line)):
                if(line[i] == '?'):
                    flage+=1
            if flage == 0:
                csvfile.write(line[:-2]+'\n')

with open('adult/adult_after.data','w', encoding='utf-8') as csvfile:
    with open('adult/adult.data', 'r', newline='') as filein:
        for line in filein:
            flage = 0
            for i in range(0,len(line)):
                if(line[i] == '?'):
                    flage+=1
            if flage == 0:
                csvfile.write(line)

In [3]:
def regression_model_build(Xlen):
    model = models.Sequential()
    model.add(Dense(80, activation='relu', input_dim=Xlen))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae','mape'])
    return model

In [4]:
adult = pd.read_csv('adult/adult_after.data', header=None)
adult_test = pd.read_csv('adult/adult_after.test', skiprows=[0], header=None)

reg_x = adult.iloc[:,:]
reg_x.pop(12)
reg_y = adult.iloc[:,12]

test_x_reg = adult_test.iloc[:,:]
test_x_reg.pop(12)
test_y_reg = adult_test.iloc[:,12]

ohe_x = OneHotEncoder(handle_unknown='ignore')
ohe_x.fit(reg_x)
reg_X_train = ohe_x.transform(reg_x).toarray()
reg_X_test = ohe_x.transform(test_x_reg).toarray()

In [5]:
reg_mld = regression_model_build(len(reg_X_train[0]))
reg_mld.fit(reg_X_train,reg_y, epochs = 100, batch_size = 100)

Epoch 1/100
302/302 [==============================] - 11s 30ms/step - loss: 174.9181 - mae: 9.6140 - mape: 32.1422
Epoch 2/100
302/302 [==============================] - 9s 30ms/step - loss: 131.9499 - mae: 8.4471 - mape: 28.8952
Epoch 3/100
302/302 [==============================] - 9s 28ms/step - loss: 118.4190 - mae: 7.8730 - mape: 27.1101
Epoch 4/100
302/302 [==============================] - 9s 30ms/step - loss: 107.8921 - mae: 7.4000 - mape: 25.4748
Epoch 5/100
302/302 [==============================] - 9s 29ms/step - loss: 99.5917 - mae: 7.0903 - mape: 24.1560
Epoch 6/100
302/302 [==============================] - 9s 29ms/step - loss: 90.5387 - mae: 6.7531 - mape: 22.7136
Epoch 7/100
302/302 [==============================] - 9s 29ms/step - loss: 82.6982 - mae: 6.4597 - mape: 21.4535
Epoch 8/100
302/302 [==============================] - 9s 29ms/step - loss: 73.6170 - mae: 6.0677 - mape: 19.9445
Epoch 9/100
302/302 [==============================] - 9s 30ms/step - loss: 67.5055

In [6]:
test_loss, test_mae, test_mape = reg_mld.evaluate(reg_X_test, test_y_reg)

471/471 [==============================] - 2s 5ms/step - loss: 130.8461 - mae: 7.7768 - mape: 26.8496


In [7]:
print('loss:', test_loss)
print('mae:', test_mae)
print('mape:', test_mape)

loss: 130.8461456298828
mae: 7.776756763458252
mape: 26.849597930908203
